In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Goal
The objective of this notebook is to:
1. describe the __motivation__ behind developing hypergrids,
1. describe the __requirements__,
1. explain __how__ we implemented them,
1. and to __demonstrate hypergrid usage__.

# Intended Audience

This document has been authored for the benefit of all users of and contributors to the MLOS project.

# Motivation

## Describing search space is fundamental
Every optimizer we have examined (SMAC, Hypermapper, bayes_opt, scipy) requires that we describe the search space for the optimizer to explore. They all differed in the exact representation and functionality but it quickly became apparent that the description of the search space (or the domain) is a fundamental part of any optimizers API.

__Requirement 1__: Hypergrids are meant to provide a superset of the union of the functionality provided by the aforementioned optimizers' search spaces. We also need to be able to build adapters that would translate a hypergrid into each optimizers search space representation.

## Hypergrids usage

Hypergrids are used throughout the optimization process. 

### Smart component
Smart components are parameterizable. The value for each parameter belongs to a set of legal values - forming one of the dimensions of the search space - let's call that a _parameter dimension_. Thus, the search space for the entire component is a subset of the cross product of all its _parameter dimensions_.

__Requirement 2__: Hypergrids must provide an efficient way of representing a componet's parameter search space.

Nota Bene: A smart component might provide multiple alternative and mutually exclusive implementations of data structures or algorithms (e.g. spinlocks have Exponential Backoff and Optimistic _algorithmType_). These alternatives are further configured by disjoint sets of parameters. Which set of parameters is meaningful is determined by the "switch" parameter. 

In the case of smart spinlocks: choosing the "Optimistic" algorithm makes the _minSpinCount_ and _maxSpinCount_ parameters meaningless, but _aquireSpinCount_ meaningful. Thus the _algorithmType_ is a switch that toggles between those two sets of parameters.

This yields our next requirements:

__Requirement 3__: Hypergrids must provide an efficient way to describe disjoint parameter spaces.

__Requirement 4__: Hypergrids must allow to specify constrains between parameters. For example: ```minSpinCount < maxSpinCount```, or even: ```minSpinCount < maxSpinCount / 10```.

### Target system
MLOS enabled systems (such as SQL Server) will in general contain multiple smart components. Each smart component's search space is described by its own Hypergrid. It's possible that two completely unrelated components will have parameters with the same name (e.g. "n", "size", or "timeout"). The entire system's search space is a subset of the cartesian product of its constituent smart components' Hypergrids.


__Requirement 5__: Hypergrids must handle parameter name collisions between different components in the same system.

__Requirement 6__: Hypergrids must efficiently represent a cartesian product of two or more Hypergrids.


### Optimization problem description
Each optimization problem is described by a parameter space (a Hypergrid), a context space (a Hypergrid), an objective space (a Hypergrid) among other things. 

### Bayesian Optimizer
The MLOS Bayesian Optimizer uses Hypergrids throughout its stack:
1. In the surrogate models - Hypergrids describe each model's input and output spaces.
1. In the utility function optimizer - Hypergrids describe the set of legal configurations that this optimizer can suggest. Hypergrids are also used to check if any given configuration is legal.
1. In the random config generation - Hypergrids are used to efficiently generate random legal configurations.

__Requirement 7__: Hypergrids must efficiently determine if a given configuration is legal.

__Requirement 8__: Hypergrids must support efficient generation of random configurations according to a distribution specified by the caller.

### Model Tomograph
Model Tomograph is used to display the internal state of the model over time. It uses Hypergrids to determine the axes, scales and labels for heatmaps describing any two dimensions. It also uses Hypergrids to generate linspaces (and meshgrids) along each dimension (and each pair of dimensions).

__Requirement 9__: Hypergrids must support efficient generation of linspaces and meshgrids.

### A note on efficiency
We have stressed efficiency in almost all of the requirements. Hypergrids are fundamental to many aspects of the Bayesian Optimizer's operations and functions such as random() or \__contains\__() can be invoked thousands or millions of times per iteration - almost always on the critical path.


# Implementation

## Sets, sets, sets
A value of a given parameter belongs to a __set__ of legal values. 
All legal configurations for a component are a __subset__ of the __cartesian product__ of each parameter's __set__ of legal values.
For any given parameter value or configuration we must check if it belongs to a __set__ of legal values.
We must be able to generate a random configuration from a __set__ of legal configurations.

The idea of sets and set algebra pervades our requirements through and through. Consequently, it also pervades our implementation.

# Dimensions

## Dimensions are sets

Each parameter can assume a value that belongs to a set of legal values. We represet such a set by one of the following Dimension classes:

* ContinuousDimension - to describe continuous domains.
* DiscreteDimension - should be renamed to IntegerDimension.
* CategoricalDimension - represents an unordered set of discrete legal values (e.g. colors: {red, blue, black})
* OrdinalDimension - represents an ordered set of discrete legal values (e.g. {bad, medium, good})

Each of them implements set operations: union, difference, intersection. The results of such operations can further give rise to:
* CompositeDimension
* EmptyDimension

## Dimension Usage

The following examples demonstrate construction and basic usage of the various Dimension classes. Further examples can be found in `<MLOS_ROOT>/source/Mlos.Python/mlos/Spaces/unit_tests`.

In [2]:
from mlos.Spaces import ContinuousDimension, DiscreteDimension, OrdinalDimension, CategoricalDimension

#### Continuous [0, 10] dimension

In [3]:
zero_to_ten = ContinuousDimension(name='x', min=0, max=10)
print(zero_to_ten, "<- note the square brackets '[]', they indicate that the endpoints are in the set.")

x: [0.00, 10.00] <- note the square brackets '[]', they indicate that the endpoints are in the set.


In [4]:
assert 0 in zero_to_ten

In [5]:
assert 5 in zero_to_ten

In [6]:
assert 11 not in zero_to_ten

#### Continuous (1, 5) dimension

In [7]:
one_to_five_exclusive = ContinuousDimension(name='x', min=1, max=5, include_min=False, include_max=False)
print(one_to_five_exclusive, "<- note the parentheses '()', they indicate that the endpoints are not in the set.")

x: (1.00, 5.00) <- note the parentheses '()', they indicate that the endpoints are not in the set.


In [8]:
assert 1 not in one_to_five_exclusive

In [9]:
assert 3 in one_to_five_exclusive

In [10]:
assert 5 not in one_to_five_exclusive

#### Set operations on continuous dimensions

In [11]:
# Containment.
#
assert one_to_five_exclusive in zero_to_ten

In [12]:
zero_to_ten.difference(one_to_five_exclusive)

x: [0.00, 1.00] UNION [5.00, 10.00]

In [13]:
zero_to_ten.union(one_to_five_exclusive)

x: [0.00, 10.00]

In [14]:
zero_to_ten.intersection(one_to_five_exclusive)

x: (1.00, 5.00)

Set operations can be chained:

In [15]:
some_complicated_dimension = ContinuousDimension(name='x', min=0, max=10).union(
    ContinuousDimension(name='x', min=20, max=30)
).union(
    ContinuousDimension(name='x', min=40, max=50)
).difference(
    ContinuousDimension(name='x', min=1, max=5)
).difference(
    ContinuousDimension(name='x', min=21, max=25)
).intersection(
    ContinuousDimension(name='x', min=0, max=50, include_min=False, include_max=False)
)
print(some_complicated_dimension)

x: (0.00, 1.00) UNION (5.00, 10.00] UNION [20.00, 21.00) UNION (25.00, 30.00] UNION [40.00, 50.00)


In [16]:
assert 0 not in some_complicated_dimension

In [17]:
assert 0.5 in some_complicated_dimension

In [18]:
assert 45 in some_complicated_dimension

#### Generating random value from dimension
For a continuous dimension we can easily generate a random value. Note that the distribution from which we draw the random value is not an inherent property of the dimension. Right now, the call to .random() returns draws from a uniform distribution over all values in the dimension, but down the road we plan to add an argument to the .random() function so that the caller can specify the distribution.

In [19]:
[zero_to_ten.random() for _ in range(10)]

[3.5766689247123997,
 1.5696057340100456,
 4.552937576298108,
 8.54933294213393,
 6.082081676460877,
 3.35230235202159,
 0.9831877589704241,
 8.886239710348544,
 7.843159341455794,
 0.7968869903132036]

However, note that `some_complicated_dimension` is in fact a union of multiple chunks:

In [20]:
print(some_complicated_dimension)

x: (0.00, 1.00) UNION (5.00, 10.00] UNION [20.00, 21.00) UNION (25.00, 30.00] UNION [40.00, 50.00)


It's called a composite dimension and is a result of union, difference, and intersection operations on continuous dimensions.

However, random number generation in composite dimension has not been implemented yet.

In [21]:
try:
    some_complicated_dimension.random()
except NotImplementedError as e:
    print("Not implemented!")

Not implemented!


It is neither difficult nor urgent to implement. 

The complication arises, because a composite dimension is built atop an interval tree of ContinuousDimension objects, referred to in the code as chunks. To sample uniformly from such a composite dimension, we must know the sizes of each chunk and sample from each chunk with a probability proportional to its size.

### Discrete Dimension

Represents a set of integers.

In [22]:
one_two = DiscreteDimension(name='x', min=1, max=2)
one_two_three = DiscreteDimension(name='x', min=1, max=3)
ten_to_twenty = DiscreteDimension(name='x', min=10, max=20)
one_to_hundred = DiscreteDimension(name='x', min=1, max=100)

print(one_two)
print(one_two_three)
print(ten_to_twenty)
print(one_to_hundred)

x: {1, 2}
x: {1, 2, 3}
x: {10, 11, ... , 20}
x: {1, 2, ... , 100}


In [23]:
assert 1 in one_two
assert 1 in one_two_three
assert 5 in one_to_hundred
assert 5 not in one_two
assert 5 not in ten_to_twenty
assert 15 in ten_to_twenty

#### Set operations on discrete dimensions

In [24]:
one_two.union(ten_to_twenty)

x: {1, 2} UNION {10, 11, ... , 20}

In [25]:
assert 15 in one_two.union(ten_to_twenty)

In [26]:
one_to_hundred.difference(ten_to_twenty)

x: {1, 2, ... , 9} UNION {21, 22, ... , 100}

In [27]:
assert 15 not in one_to_hundred.difference(ten_to_twenty)
assert 50 in one_to_hundred.difference(ten_to_twenty)

In [28]:
one_to_hundred.intersection(ten_to_twenty)

x: {10, 11, ... , 20}

In [29]:
assert one_to_hundred.intersection(ten_to_twenty) == ten_to_twenty

#### Generating random value from discrete dimension

In [30]:
print([one_to_hundred.random() for _ in range(10)])

[39, 1, 16, 69, 27, 2, 8, 56, 78, 58]


### CategoricalDimension

In [31]:
# Create a dimension corresponding to our algorithm type
#
spinlock_algorithm_type_dimension = CategoricalDimension(name="spinlock_algorithm_type", values=["Optimistic", "ExponentialBackOff"])

# Create a few random parameter values
#
num_values = 10
print(f"Generating {num_values} random algorithm values:")
for i in range(num_values):
    print(f"\t[{i+1}/{num_values}] Random algorithm type: {spinlock_algorithm_type_dimension.random()}")

Generating 10 random algorithm values:
	[1/10] Random algorithm type: Optimistic
	[2/10] Random algorithm type: Optimistic
	[3/10] Random algorithm type: ExponentialBackOff
	[4/10] Random algorithm type: Optimistic
	[5/10] Random algorithm type: ExponentialBackOff
	[6/10] Random algorithm type: Optimistic
	[7/10] Random algorithm type: Optimistic
	[8/10] Random algorithm type: Optimistic
	[9/10] Random algorithm type: Optimistic
	[10/10] Random algorithm type: Optimistic


### Ordinal Dimension

Ordinal dimension subclasses from CategoricalDimension and the only difference is that it maintains ordering. The reason for its existence is that some models can leverage the fact that values in the set are ordered. For example: for categorical dimensions (unordered) we sometimes perform one-hot-encoding before feeding them into the model. For ordinal dimensions however, we may achieve better results by simply assigning monotonically increasing integers to values in the set.

In [32]:
quality = OrdinalDimension(name='quality', ordered_values=['good', 'better', 'best'])
difficulty = OrdinalDimension(name='difficulty', ordered_values=['easy', 'hard', 'np-complete'])
print(quality)
print(difficulty)

quality: {good, better, best}
difficulty: {easy, hard, np-complete}


# Hypergrids
A simple hypergrid is a representation of a search space. It is a cartesian product of dimensions, and therefore also a set.

A few words on the etymology:
* hyper - for hyperdimensional, like hypercubes and hyperplanes
* grid - if we only allowed continuous dimensions, the word hypercube would aptly describe the set of allowed configurations. But since we allow for Discrete (Integer), Categorical, and Ordinal dimensions, we end up with a hyperdimensional grid. 

Ergo: hypergrid.

<sub>In the name of completeness, let us also point out that a hierarchical hypergrid is only a subset of the cartesian product of all of its dimensions.</sub>

## Flat Hypergrids

First, let's import a flat hypergrid with several dimensions. This document has nothing to do with glowworm swarm optimization, we are merely using this hypergrid as an example.

In [33]:
from mlos.Optimizers.ExperimentDesigner.UtilityFunctionOptimizers.GlowWormSwarmOptimizer import glow_worm_swarm_optimizer_config_store
print(glow_worm_swarm_optimizer_config_store.parameter_space)

  Name: glow_worm_swarm_optimizer_config
  Dimensions:
    num_initial_points_multiplier: {1, 2, ... , 10}
    num_worms: {1, 2, ... , 1000}
    num_iterations: {1, 2, ... , 1000}
    luciferin_decay_constant: [0.00, 1.00]
    luciferin_enhancement_constant: [0.00, 1.00]
    step_size: [0.00, 1.00]
    initial_decision_radius: (0.00, 1.00]
    max_sensory_radius: [0.50, 10.00]
    desired_num_neighbors: {1, 2, ... , 100}
    decision_radius_adjustment_constant: [0.00, 1.00]


### Defining a flat Hypergrid
... is as simple as providing its name and the dimensions comprising it. Let's reproduce the hypergrid from above.

In [34]:
from mlos.Spaces import Point, SimpleHypergrid

glow_worm_config_space = SimpleHypergrid(
    name="glow_worm_swarm_optimizer_config",
    dimensions=[
        DiscreteDimension(name="num_initial_points_multiplier", min=1, max=10),
        DiscreteDimension(name="num_worms", min=1, max=1000),
        DiscreteDimension(name="num_iterations", min=1, max=1000),
        ContinuousDimension(name="luciferin_decay_constant", min=0, max=1),
        ContinuousDimension(name="luciferin_enhancement_constant", min=0, max=1),
        ContinuousDimension(name="step_size", min=0, max=1),  # TODO: make this adaptive
        ContinuousDimension(name="initial_decision_radius", min=0, max=1, include_min=False),
        ContinuousDimension(name="max_sensory_radius", min=0.5, max=10),
        DiscreteDimension(name="desired_num_neighbors", min=1, max=100),
        ContinuousDimension(name="decision_radius_adjustment_constant", min=0, max=1)
    ]
)

In [35]:
print(glow_worm_config_space)

  Name: glow_worm_swarm_optimizer_config
  Dimensions:
    num_initial_points_multiplier: {1, 2, ... , 10}
    num_worms: {1, 2, ... , 1000}
    num_iterations: {1, 2, ... , 1000}
    luciferin_decay_constant: [0.00, 1.00]
    luciferin_enhancement_constant: [0.00, 1.00]
    step_size: [0.00, 1.00]
    initial_decision_radius: (0.00, 1.00]
    max_sensory_radius: [0.50, 10.00]
    desired_num_neighbors: {1, 2, ... , 100}
    decision_radius_adjustment_constant: [0.00, 1.00]


In [36]:
# Let's convince ourselves that they are the same.
#
assert str(glow_worm_config_space) == str(glow_worm_swarm_optimizer_config_store.parameter_space)

#### Generating a random point from a hypergrid
Lot's of components throughout the code depend our ability to efficiently generate random, valid configurations. Hypergrids provide this functionality for individual points, and for entire dataframes.


In [37]:
# Single point - converting to json makes it easier to read.
#
random_point = glow_worm_config_space.random()
print(type(random_point))
print(random_point.to_json(indent=2))

<class 'mlos.Spaces.Point.Point'>
{
  "num_initial_points_multiplier": 2,
  "num_worms": 358,
  "num_iterations": 308,
  "luciferin_decay_constant": 0.9900770752974355,
  "luciferin_enhancement_constant": 0.9223858636961079,
  "step_size": 0.09008589662723443,
  "initial_decision_radius": 0.4513730247648293,
  "max_sensory_radius": 7.6104082050811215,
  "desired_num_neighbors": 92,
  "decision_radius_adjustment_constant": 0.39733626710091496
}


#### Generating Random DataFrames
Since most of the optimizer's components can __operate on batches of data much more efficiently than on individual points__.

In [38]:
# Entire dataframe
#
glow_worm_config_space.random_dataframe(num_samples=10)

,num_initial_points_multiplier,num_worms,num_iterations,luciferin_decay_constant,luciferin_enhancement_constant,step_size,initial_decision_radius,max_sensory_radius,desired_num_neighbors,decision_radius_adjustment_constant
0,3,858,16,0.687969,0.305402,0.767388,0.028630,4.658216,98,0.237023
1,4,97,791,0.106594,0.986211,0.038526,0.399608,5.160340,41,0.734487
2,3,446,737,0.465471,0.853240,0.403752,0.660720,0.810461,57,0.526858
3,4,171,857,0.344040,0.186406,0.151722,0.299323,4.606827,4,0.028664
4,8,718,800,0.952037,0.869736,0.957513,0.578656,9.504716,5,0.492985
5,10,710,52,0.286364,0.526632,0.742756,0.646780,1.208530,15,0.736332
6,1,393,601,0.981068,0.042705,0.475467,0.492001,0.739042,33,0.529902
7,3,221,465,0.448932,0.040731,0.274861,0.616297,8.307484,9,0.095790
8,2,559,204,0.247446,0.587054,0.914082,0.258602,2.372909,77,0.431750
9,10,20,435,0.256607,0.286974,0.853695,0.252879,2.840333,68,0.249751


##### A note on distribution
In a flat hypergrid the points are sampled uniformly across the entire hypergrid.

### Testing containment

A very useful feature is to test whether a given point belongs to a hypergrid or not. An ability to test whether a dataframe belongs to a hypergrid might be useful, but we have not seen the need to implement it yet.

In [39]:
valid_point = glow_worm_config_space.random()
assert valid_point in glow_worm_config_space

In [40]:
# And now let's construct an invalid point.
#
invalid_point = valid_point.copy()

# This dimension has to be positive so let's set it to zero.
#
invalid_point.num_initial_points_multiplier = 0

# And finally test.
#
assert invalid_point not in glow_worm_config_space

##### Converting between a Point and a DataFrame
A point can be trivially converted to a DataFrame. The reverse is also possible for DataFrames of length 1.  

In [41]:
valid_point_df = valid_point.to_dataframe()
valid_point_df

,num_initial_points_multiplier,num_worms,num_iterations,luciferin_decay_constant,luciferin_enhancement_constant,step_size,initial_decision_radius,max_sensory_radius,desired_num_neighbors,decision_radius_adjustment_constant
0,1,152,555,0.153657,0.377911,0.317738,0.569483,9.303174,76,0.128463


In [42]:
reconstructed_valid_point = Point.from_dataframe(valid_point_df)
print(reconstructed_valid_point.to_json(indent=2))

{
  "num_initial_points_multiplier": 1,
  "num_worms": 152,
  "num_iterations": 555,
  "luciferin_decay_constant": 0.15365722568855145,
  "luciferin_enhancement_constant": 0.37791076794294753,
  "step_size": 0.31773792022806624,
  "initial_decision_radius": 0.5694825895867698,
  "max_sensory_radius": 9.303174124340375,
  "desired_num_neighbors": 76,
  "decision_radius_adjustment_constant": 0.12846263991018636
}


#### Equality between two points


In [43]:
assert valid_point == reconstructed_valid_point

In [44]:
assert valid_point != invalid_point

# Hierarchical Hypergrids

Flat Hypergrids are powerful, but we can do better. Consider a situation when we want to ask the optimizer to choose one of the available algorithm implementations, and then optimize its parameters. Such multi-level optimization is pretty hard to begin with so we need to give the optimizer every advantage we can afford. This is where hierarchical hypergrids come into picture. They allow the developer to group parameters by the the implementation they pertain to.  

### What the world looks like without hierarchical hypergrids

One of our internal use-cases for MLOS is an optimization of spinlocks. We have two alternative back off policies: Optimistic, and ExponentialBackOff. They are governed by two distinct set of parameters. If we were forced to use a flat hypergrid we would have to describe them as follows:

In [45]:
flat_spinlock_parameter_space = SimpleHypergrid(
    name="SpinlockParameterSpace",
    dimensions = [
        CategoricalDimension(name="backOffPolicy", values=["Optimistic", "ExponentialBackOff"]),
        DiscreteDimension(name="shortBackOffMilliSeconds", min=1, max=1 << 15),
        DiscreteDimension(name="longBackOffMilliSeconds", min=1, max=1 << 15),
        DiscreteDimension(name="longBackOffWaitMilliSeconds", min=1, max=1 << 15),
        DiscreteDimension(name="minSpinCount", min=1, max=1 << 20),
        DiscreteDimension(name="maxSpinCount", min=1, max=1 << 20),
        DiscreteDimension(name="maxBackOffAttempts", min=1, max=1 << 20),
        DiscreteDimension(name="acquireSpinCount", min=1, max=1 << 20)
    ]
)
print(flat_spinlock_parameter_space)
flat_spinlock_parameter_space.random_dataframe(num_samples=10)

  Name: SpinlockParameterSpace
  Dimensions:
    backOffPolicy: {Optimistic, ExponentialBackOff}
    shortBackOffMilliSeconds: {1, 2, ... , 32768}
    longBackOffMilliSeconds: {1, 2, ... , 32768}
    longBackOffWaitMilliSeconds: {1, 2, ... , 32768}
    minSpinCount: {1, 2, ... , 1048576}
    maxSpinCount: {1, 2, ... , 1048576}
    maxBackOffAttempts: {1, 2, ... , 1048576}
    acquireSpinCount: {1, 2, ... , 1048576}


,backOffPolicy,shortBackOffMilliSeconds,longBackOffMilliSeconds,longBackOffWaitMilliSeconds,minSpinCount,maxSpinCount,maxBackOffAttempts,acquireSpinCount
0,ExponentialBackOff,2946,12579,6304,187462,87515,284522,993971
1,Optimistic,13864,13366,24815,328306,691362,107536,39941
2,ExponentialBackOff,22921,7388,32139,632675,702987,705257,458266
3,ExponentialBackOff,11956,11878,1418,699402,571799,60422,974550
4,Optimistic,30735,8617,372,429390,963677,134452,829088
5,ExponentialBackOff,19018,12011,18490,857830,767457,967221,832317
6,ExponentialBackOff,27950,15199,18661,791411,258958,156595,379946
7,ExponentialBackOff,27957,14241,6528,291060,646112,912766,15046
8,ExponentialBackOff,15492,9594,26207,178198,424347,769105,69708
9,Optimistic,9884,11310,7888,653120,466989,802738,192278


We can see that the configuration parameters are mixed for both the implementations. With sufficient training, the surrogate model within the optimizer might discover which parameters pertain to which policy, but it's inefficient. The unused parameters, effectively serve as noise.

### Using a hierarchical hypergrid to partition the search space
A superior alternative is to partition the search space and group the parameters into subspaces. We can do it, by constructing a hierarchical hypergrid.

In [46]:
root_spinlock_parameter_space = SimpleHypergrid(
    name="SpinlockParameterSpace",
    dimensions=[
        CategoricalDimension(name="backOffPolicy", values=["Optimistic", "ExponentialBackOff"])
    ]
)
print(root_spinlock_parameter_space)

  Name: SpinlockParameterSpace
  Dimensions:
    backOffPolicy: {Optimistic, ExponentialBackOff}


In the top level of the hierarchy we specify only the parameter controlling the back-off policy. Let's now specify a subgrid for each of the policies:

In [47]:
optimistic_backoff_parameter_space = SimpleHypergrid(
    name="OptimisticBackoff",
    dimensions=[
        DiscreteDimension(name="acquireSpinCount", min=1, max=1 << 20),
    ]
)
print(optimistic_backoff_parameter_space)

  Name: OptimisticBackoff
  Dimensions:
    acquireSpinCount: {1, 2, ... , 1048576}


In [48]:
exponential_backoff_parameter_space = SimpleHypergrid(
    name="ExponentialBackoff",
    dimensions=[
        DiscreteDimension(name="minSpinCount", min=1, max=1 << 20),
        DiscreteDimension(name="maxSpinCount", min=1, max=1 << 20),
        DiscreteDimension(name="maxBackOffAttempts", min=1, max=1 << 20),
        DiscreteDimension(name="shortBackOffMilliSeconds", min=1, max=1 << 15),
        DiscreteDimension(name="longBackOffMilliSeconds", min=1, max=1 << 15),
        DiscreteDimension(name="longBackOffWaitMilliSeconds", min=1, max=1 << 15),
    ]
)
print(exponential_backoff_parameter_space)

  Name: ExponentialBackoff
  Dimensions:
    minSpinCount: {1, 2, ... , 1048576}
    maxSpinCount: {1, 2, ... , 1048576}
    maxBackOffAttempts: {1, 2, ... , 1048576}
    shortBackOffMilliSeconds: {1, 2, ... , 32768}
    longBackOffMilliSeconds: {1, 2, ... , 32768}
    longBackOffWaitMilliSeconds: {1, 2, ... , 32768}


Joining the three spaces together to form a hierarchy can be accomplished by specifying a join dimension between them. The meaning of the join dimensions is best explained by the output below:

In [49]:
hierarchical_spinlock_parameter_space = root_spinlock_parameter_space.join(
    subgrid=optimistic_backoff_parameter_space,
    on_external_dimension=CategoricalDimension(name="backOffPolicy", values=["Optimistic"])
).join(
    subgrid=exponential_backoff_parameter_space,
    on_external_dimension=CategoricalDimension(name="backOffPolicy", values=["ExponentialBackOff"])
)
print(hierarchical_spinlock_parameter_space)

  Name: SpinlockParameterSpace
  Dimensions:
    backOffPolicy: {Optimistic, ExponentialBackOff}

  IF backOffPolicy IN {ExponentialBackOff} THEN (
    Name: ExponentialBackoff
    Dimensions:
      minSpinCount: {1, 2, ... , 1048576}
      maxSpinCount: {1, 2, ... , 1048576}
      maxBackOffAttempts: {1, 2, ... , 1048576}
      shortBackOffMilliSeconds: {1, 2, ... , 32768}
      longBackOffMilliSeconds: {1, 2, ... , 32768}
      longBackOffWaitMilliSeconds: {1, 2, ... , 32768}
  )

  IF backOffPolicy IN {Optimistic} THEN (
    Name: OptimisticBackoff
    Dimensions:
      acquireSpinCount: {1, 2, ... , 1048576}
  )


A useful side effect of this arrangement is that alternative implementations of smart components (TODO: define a smart component) can be __developed separately, but optimized jointly__. And in general we can compose a larger system from many smart components. If you study the Optimizer code, you will find this principle in action.

Furthermore, if the optimizer uses an ensemble model, it can now __logically partition the input features to consituent models__. This is exactly what we do in the Homogeneous Random Forest Regression Model. Each decision tree gets a semantically coherent subset of all parameters. This is doubly nice, because no observation can ever simultanously contain valid parameters for both implementations, so we avoid a need to impute.

_A few words on the history and geography of hypergrids:_ At one point we used to have two classes: SimpleHypergrid and CompositeHypergrid to represent the flat and hierarchical spaces respectively. We have since simplified the design and the SimpleHypergrid can now represent both. Suitable renames are upcoming.

_A note on the mathematical interpretation of this join:_ Observant readers may notice that this use of the .join() operator is tantamount to producing a disjoint union of the sets described by each policy's subgrid. Curious readers may consider a possiblity of using .join() operators as a simple way to implement a cartesian product of two spaces.

### Operating on hierarchical hypergrids
All of the functions described above for the flat hypergrid, work also for hierarchical hypergrids. Afterall, they are all instances of the SimpleHypergrid class. Let's demonstrate them and point out some noteworthy behavior changes.

#### Generating a random point from a hierarchical hypergrid
We really have to generate a few to see the important behavior change:

In [50]:
for _ in range(5):
    print(hierarchical_spinlock_parameter_space.random().to_json(indent=2))
    print("-----------------------------------------------")

{
  "backOffPolicy": "Optimistic",
  "OptimisticBackoff.acquireSpinCount": 985983
}
-----------------------------------------------
{
  "backOffPolicy": "ExponentialBackOff",
  "ExponentialBackoff.minSpinCount": 73828,
  "ExponentialBackoff.maxSpinCount": 355929,
  "ExponentialBackoff.maxBackOffAttempts": 219200,
  "ExponentialBackoff.shortBackOffMilliSeconds": 25369,
  "ExponentialBackoff.longBackOffMilliSeconds": 31964,
  "ExponentialBackoff.longBackOffWaitMilliSeconds": 4143
}
-----------------------------------------------
{
  "backOffPolicy": "ExponentialBackOff",
  "ExponentialBackoff.minSpinCount": 711557,
  "ExponentialBackoff.maxSpinCount": 427291,
  "ExponentialBackoff.maxBackOffAttempts": 850009,
  "ExponentialBackoff.shortBackOffMilliSeconds": 4693,
  "ExponentialBackoff.longBackOffMilliSeconds": 1733,
  "ExponentialBackoff.longBackOffWaitMilliSeconds": 4394
}
-----------------------------------------------
{
  "backOffPolicy": "ExponentialBackOff",
  "ExponentialBackoff.mi

Note that each point is valid and contains no superfluous parameters. Contrast this with the point generated from our flat hypergrid:

In [51]:
print(flat_spinlock_parameter_space.random().to_json(indent=2))

{
  "backOffPolicy": "Optimistic",
  "shortBackOffMilliSeconds": 26983,
  "longBackOffMilliSeconds": 23970,
  "longBackOffWaitMilliSeconds": 32342,
  "minSpinCount": 206419,
  "maxSpinCount": 945646,
  "maxBackOffAttempts": 1014201,
  "acquireSpinCount": 514595
}


Now every model that would try to learn something about the optimistic back off, also has to sort through 6 dimensions worth of noise.

#### Generating an entire DataFrame

In [52]:
hierarchical_spinlock_parameter_space.random_dataframe(num_samples=10)

,backOffPolicy,ExponentialBackoff.minSpinCount,ExponentialBackoff.maxSpinCount,ExponentialBackoff.maxBackOffAttempts,ExponentialBackoff.shortBackOffMilliSeconds,ExponentialBackoff.longBackOffMilliSeconds,ExponentialBackoff.longBackOffWaitMilliSeconds,OptimisticBackoff.acquireSpinCount
0,ExponentialBackOff,624011.0,185710.0,305820.0,14740.0,5001.0,15854.0,NaN
1,ExponentialBackOff,934135.0,11504.0,533069.0,18924.0,10606.0,26984.0,NaN
2,Optimistic,NaN,NaN,NaN,NaN,NaN,NaN,714496.0
3,Optimistic,NaN,NaN,NaN,NaN,NaN,NaN,595904.0
4,ExponentialBackOff,879032.0,510669.0,353036.0,24833.0,28065.0,29145.0,NaN
5,Optimistic,NaN,NaN,NaN,NaN,NaN,NaN,225831.0
6,ExponentialBackOff,313113.0,178714.0,676878.0,15046.0,228.0,22526.0,NaN
7,ExponentialBackOff,22722.0,809518.0,633316.0,10381.0,650.0,14089.0,NaN
8,ExponentialBackOff,426729.0,699243.0,396577.0,5198.0,28819.0,19322.0,NaN
9,ExponentialBackOff,322754.0,786872.0,607471.0,23484.0,18472.0,6063.0,NaN


See all those NaN values? That's numpy's equivalent of a null (Not a Number). Now we are generating the data in a way that helps our models converge, rather than creating extra noise for them to sift through.

##### A note on distribution

In a hierarchical hypergrid points are sampled uniformly across the root hypergrid. Then, we evaluate which of the subgrids were 'activated' and recursively repeat the procedure within them. This does not result in a uniform distribution throughout the entire space, but rather we uniformly sample from the available implementations, and for the selected implementation, we uniformly sample from its parameters.

The effect of this scheme is that simpler implementations are explored more thoroughly. So in effect we reward simplicity. The uniform sampling across the entire space would favour complex subgrids with many parameters. Such spaces are larger, and thus to maintain uniformity we would have to sample from them more often.

We realize that this default behavior will not suit every purpose, so down the road we will provide a way to control this behavior.

Finally, all samples are still independent and identically distributed (i.i.d.). Just not uniformly.

#### Testing containment

In [53]:
valid_point = hierarchical_spinlock_parameter_space.random()
print(valid_point.to_json(indent=2))
assert valid_point in hierarchical_spinlock_parameter_space 
print(valid_point in hierarchical_spinlock_parameter_space)

{
  "backOffPolicy": "Optimistic",
  "OptimisticBackoff.acquireSpinCount": 752605
}
True


In [54]:
invalid_point = valid_point.copy()
invalid_point.backOffPolicy = "Reluctant"
print(invalid_point.to_json(indent=2))
assert invalid_point not in hierarchical_spinlock_parameter_space
print(invalid_point in hierarchical_spinlock_parameter_space)

{
  "backOffPolicy": "Reluctant",
  "OptimisticBackoff.acquireSpinCount": 752605
}
False


#### Converting between a Point and a DataFrame
Works mostly the same, but since a Point does not carry information about which Space it came from, it will produce a dense DataFrame with no NaNs and no columns for NaNs because all dimensions this Point is aware of are valid.

In [55]:
random_df = hierarchical_spinlock_parameter_space.random_dataframe(num_samples=1)
random_df

,backOffPolicy,OptimisticBackoff.acquireSpinCount
0,Optimistic,824423


In [56]:
yet_another_random_point = Point.from_dataframe(random_df)
print(yet_another_random_point.to_json(indent=2))

{
  "backOffPolicy": "Optimistic",
  "OptimisticBackoff.acquireSpinCount": 824423
}


In [57]:
valid_point_df = valid_point.to_dataframe()
valid_point_df

,backOffPolicy,OptimisticBackoff.acquireSpinCount
0,Optimistic,752605


#### Comparison between points works as before

In [58]:
reconstructed_valid_point = Point.from_dataframe(valid_point_df)
assert valid_point == reconstructed_valid_point

In [59]:
assert valid_point != yet_another_random_point

## HypergridAdapters

Bijective (most often) functions used to preprocess the data before passing it on to the next component in the pipeline.

### The need for HypergridAdapters

#### Intended audience
This section is indented primarily for the consumption by developers contributing to the Optimizer. HypergridAdapters remain an internal mechanism to provide combatility between components and thus a user of the Optimizer does not need to know their mechanics in detail.

#### What we want the user experience to be?
We want to allow the users to specify the configuration space in terms closely matching how the parameters are used. So we want the users to be able to use integers, floats, strings and even booleans to define the legal values for the parameters. We further want to allow them to encode the semantic relationships between parameters by grouping them into subgrids within a hierarchical hypergrid. 

#### What the optimzier can operate on?
None of our models and only half of our utility function optimizers can operate on such complicated domains. Decision Trees need flat searchspaces with only numerical dimensions. Lasso regression, ridge regression, and RERF are similar. Glowworm swarm optimization goes further and only admits a search space where all dimensions are scaled to between 0 and 1. 

#### Bridging the gap
So we need a way to project what the user specifies (and thus all of the incoming data) to values consumable by the models. We further need to be able to project back and forth between the respective spaces of the utility function optimizers, utility functions, and the surrogate models. Hypergrid adapters are the solution we propose.

The inspiration and terminology are borrowed from The Gang of Four "Design Patterns: Elements of Reusable Object-Oriented Software". There an adapter is a structural pattern described to:
    
    Convert the interface of a class into another interface clients expect. Adapter lets classes work together that couldn't otherwise because of incompatible interfaces.
    

##### Stackable
If the parameter space specified in the optimization problem is hierarchical and has categorical dimensions, and if we want to use the glowworm swarm optimizer, we can stack several adapters to perform the required projection. The implementation makes it seamless and in this particular example almost trivial. We will take a look on how we can convert the smart spinlocks parameter space to be unit-continuous below.

##### Invertible
Almost all of the functions embedded inside of Adapters have to be bijective to be invertible. That is if we project a point P from space S, to a point P' in a space S': 
```python
p_prime = hypergrid_adapter.project_point(p)
```
We must also be able to perform the inverse of that projection:

```python
p = hypergrid_adapter.unproject_point(p_prime)
```

This is crucial if we want to be able to translate back to the user, what the optimizer came up with.

<b>One notable exception is dimensionality reduction </b> - for example principal component analysis, which is in general not invertible. In such cases, the Adapter's author must maintain a slightly weaker property: an ability to project-back any point it projected forward. Should be easy enough.

#### Let's take a look at how adapters work in practice

Firs we will convert the spinlocks search space step-by-step, and then we'll explore a convenient shortcut.

##### The long way

In [60]:
from mlos.Spaces.HypergridAdapters import CategoricalToDiscreteHypergridAdapter, DiscreteToUnitContinuousHypergridAdapter, HierarchicalToFlatHypergridAdapter

original_space = hierarchical_spinlock_parameter_space
original_df = original_space.random_dataframe(num_samples=10)
original_df

,backOffPolicy,OptimisticBackoff.acquireSpinCount,ExponentialBackoff.minSpinCount,ExponentialBackoff.maxSpinCount,ExponentialBackoff.maxBackOffAttempts,ExponentialBackoff.shortBackOffMilliSeconds,ExponentialBackoff.longBackOffMilliSeconds,ExponentialBackoff.longBackOffWaitMilliSeconds
0,Optimistic,50899.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ExponentialBackOff,NaN,159675.0,788976.0,460130.0,26737.0,10746.0,24550.0
2,ExponentialBackOff,NaN,219857.0,299668.0,371043.0,5669.0,6509.0,3354.0
3,Optimistic,640614.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ExponentialBackOff,NaN,295086.0,435846.0,526756.0,9460.0,2160.0,15894.0
5,Optimistic,877218.0,NaN,NaN,NaN,NaN,NaN,NaN
6,ExponentialBackOff,NaN,1038387.0,547279.0,347076.0,21026.0,24446.0,13138.0
7,ExponentialBackOff,NaN,433866.0,201209.0,101617.0,8295.0,19924.0,9723.0
8,Optimistic,990528.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Optimistic,32715.0,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
flattening_adapter = HierarchicalToFlatHypergridAdapter(adaptee=original_space)
flattening_adapter.project_dataframe(original_df, in_place=False)

,backOffPolicy,OptimisticBackoff___acquireSpinCount,ExponentialBackoff___minSpinCount,ExponentialBackoff___maxSpinCount,ExponentialBackoff___maxBackOffAttempts,ExponentialBackoff___shortBackOffMilliSeconds,ExponentialBackoff___longBackOffMilliSeconds,ExponentialBackoff___longBackOffWaitMilliSeconds
0,Optimistic,50899.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ExponentialBackOff,NaN,159675.0,788976.0,460130.0,26737.0,10746.0,24550.0
2,ExponentialBackOff,NaN,219857.0,299668.0,371043.0,5669.0,6509.0,3354.0
3,Optimistic,640614.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ExponentialBackOff,NaN,295086.0,435846.0,526756.0,9460.0,2160.0,15894.0
5,Optimistic,877218.0,NaN,NaN,NaN,NaN,NaN,NaN
6,ExponentialBackOff,NaN,1038387.0,547279.0,347076.0,21026.0,24446.0,13138.0
7,ExponentialBackOff,NaN,433866.0,201209.0,101617.0,8295.0,19924.0,9723.0
8,Optimistic,990528.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Optimistic,32715.0,NaN,NaN,NaN,NaN,NaN,NaN


The only difference is that the column names have been changed to no longer contain dots ('.').

In [62]:
categorical_to_discrete_adapter = CategoricalToDiscreteHypergridAdapter(adaptee=flattening_adapter)
categorical_to_discrete_adapter.project_dataframe(original_df, in_place=False)

,backOffPolicy,OptimisticBackoff___acquireSpinCount,ExponentialBackoff___minSpinCount,ExponentialBackoff___maxSpinCount,ExponentialBackoff___maxBackOffAttempts,ExponentialBackoff___shortBackOffMilliSeconds,ExponentialBackoff___longBackOffMilliSeconds,ExponentialBackoff___longBackOffWaitMilliSeconds
0,0,50899.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,159675.0,788976.0,460130.0,26737.0,10746.0,24550.0
2,1,NaN,219857.0,299668.0,371043.0,5669.0,6509.0,3354.0
3,0,640614.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NaN,295086.0,435846.0,526756.0,9460.0,2160.0,15894.0
5,0,877218.0,NaN,NaN,NaN,NaN,NaN,NaN
6,1,NaN,1038387.0,547279.0,347076.0,21026.0,24446.0,13138.0
7,1,NaN,433866.0,201209.0,101617.0,8295.0,19924.0,9723.0
8,0,990528.0,NaN,NaN,NaN,NaN,NaN,NaN
9,0,32715.0,NaN,NaN,NaN,NaN,NaN,NaN


Now each categorical value in each categorical dimension was assigned an integer. The alternative here would be to use one-hot-encoding, and the implementation of such an adapter is left as an exercise for the reader.

In [63]:
discrete_to_unit_continuous_adapter = DiscreteToUnitContinuousHypergridAdapter(adaptee=categorical_to_discrete_adapter)
unit_continuous_df = discrete_to_unit_continuous_adapter.project_dataframe(original_df, in_place=False)
unit_continuous_df

,backOffPolicy,OptimisticBackoff___acquireSpinCount,ExponentialBackoff___minSpinCount,ExponentialBackoff___maxSpinCount,ExponentialBackoff___maxBackOffAttempts,ExponentialBackoff___shortBackOffMilliSeconds,ExponentialBackoff___longBackOffMilliSeconds,ExponentialBackoff___longBackOffWaitMilliSeconds
0,0.0,0.048540,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,NaN,0.152277,0.752425,0.438813,0.815918,0.327911,0.749176
2,0.5,NaN,0.209671,0.285785,0.353853,0.172974,0.198608,0.102325
3,0.0,0.610936,NaN,NaN,NaN,NaN,NaN,NaN
4,0.5,NaN,0.281415,0.415654,0.502353,0.288666,0.065887,0.485016
5,0.0,0.836579,NaN,NaN,NaN,NaN,NaN,NaN
6,0.5,NaN,0.990282,0.521925,0.330997,0.641632,0.746002,0.400909
7,0.5,NaN,0.413766,0.191887,0.096909,0.253113,0.608002,0.296692
8,0.0,0.944640,NaN,NaN,NaN,NaN,NaN,NaN
9,0.0,0.031199,NaN,NaN,NaN,NaN,NaN,NaN


Now, each dimension was scaled down to have all values between 0 and 1. And voila, Glowworm swarm optimizer could operate on such a dataframe.

##### A shorter way
We can of course just nest the constructor calls:

In [64]:
original_to_unit_continuous_adapter = DiscreteToUnitContinuousHypergridAdapter(
    adaptee=CategoricalToDiscreteHypergridAdapter(
        adaptee=HierarchicalToFlatHypergridAdapter(
            adaptee=original_space
        )
    )
)
original_to_unit_continuous_adapter.project_dataframe(original_df, in_place=False)

,backOffPolicy,OptimisticBackoff___acquireSpinCount,ExponentialBackoff___minSpinCount,ExponentialBackoff___maxSpinCount,ExponentialBackoff___maxBackOffAttempts,ExponentialBackoff___shortBackOffMilliSeconds,ExponentialBackoff___longBackOffMilliSeconds,ExponentialBackoff___longBackOffWaitMilliSeconds
0,0.0,0.048540,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,NaN,0.152277,0.752425,0.438813,0.815918,0.327911,0.749176
2,0.5,NaN,0.209671,0.285785,0.353853,0.172974,0.198608,0.102325
3,0.0,0.610936,NaN,NaN,NaN,NaN,NaN,NaN
4,0.5,NaN,0.281415,0.415654,0.502353,0.288666,0.065887,0.485016
5,0.0,0.836579,NaN,NaN,NaN,NaN,NaN,NaN
6,0.5,NaN,0.990282,0.521925,0.330997,0.641632,0.746002,0.400909
7,0.5,NaN,0.413766,0.191887,0.096909,0.253113,0.608002,0.296692
8,0.0,0.944640,NaN,NaN,NaN,NaN,NaN,NaN
9,0.0,0.031199,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# And just to be sure
#
assert original_to_unit_continuous_adapter.project_dataframe(original_df, in_place=False).equals(unit_continuous_df)

##### The shortest way
All of the adapters implemented so far, can chain themselves.

In [66]:
adapter = DiscreteToUnitContinuousHypergridAdapter(adaptee=original_space)
projected_df = adapter.project_dataframe(original_df, in_place=False)
projected_df

,backOffPolicy,OptimisticBackoff___acquireSpinCount,ExponentialBackoff___minSpinCount,ExponentialBackoff___maxSpinCount,ExponentialBackoff___maxBackOffAttempts,ExponentialBackoff___shortBackOffMilliSeconds,ExponentialBackoff___longBackOffMilliSeconds,ExponentialBackoff___longBackOffWaitMilliSeconds
0,0.0,0.048540,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,NaN,0.152277,0.752425,0.438813,0.815918,0.327911,0.749176
2,0.5,NaN,0.209671,0.285785,0.353853,0.172974,0.198608,0.102325
3,0.0,0.610936,NaN,NaN,NaN,NaN,NaN,NaN
4,0.5,NaN,0.281415,0.415654,0.502353,0.288666,0.065887,0.485016
5,0.0,0.836579,NaN,NaN,NaN,NaN,NaN,NaN
6,0.5,NaN,0.990282,0.521925,0.330997,0.641632,0.746002,0.400909
7,0.5,NaN,0.413766,0.191887,0.096909,0.253113,0.608002,0.296692
8,0.0,0.944640,NaN,NaN,NaN,NaN,NaN,NaN
9,0.0,0.031199,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# And just to be sure
#
assert projected_df.equals(unit_continuous_df)

#### Demonstrating Invertibility
The second property that almost all adapters must have is being invertible (with the exceptions listed before).

In [68]:
unprojected_df = adapter.unproject_dataframe(projected_df, in_place=False)
unprojected_df

,backOffPolicy,OptimisticBackoff.acquireSpinCount,ExponentialBackoff.minSpinCount,ExponentialBackoff.maxSpinCount,ExponentialBackoff.maxBackOffAttempts,ExponentialBackoff.shortBackOffMilliSeconds,ExponentialBackoff.longBackOffMilliSeconds,ExponentialBackoff.longBackOffWaitMilliSeconds
0,Optimistic,50899.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ExponentialBackOff,NaN,159675.0,788976.0,460130.0,26737.0,10746.0,24550.0
2,ExponentialBackOff,NaN,219857.0,299668.0,371043.0,5669.0,6509.0,3354.0
3,Optimistic,640614.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ExponentialBackOff,NaN,295086.0,435846.0,526756.0,9460.0,2160.0,15894.0
5,Optimistic,877218.0,NaN,NaN,NaN,NaN,NaN,NaN
6,ExponentialBackOff,NaN,1038387.0,547279.0,347076.0,21026.0,24446.0,13138.0
7,ExponentialBackOff,NaN,433866.0,201209.0,101617.0,8295.0,19924.0,9723.0
8,Optimistic,990528.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Optimistic,32715.0,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# And to be sure we got back what we put in:
#
assert unprojected_df.equals(original_df)

#### Adapting Points
Works exactly the same way:

In [70]:
original_point = original_space.random()
print(original_point.to_json(indent=2))

{
  "backOffPolicy": "Optimistic",
  "OptimisticBackoff.acquireSpinCount": 286747
}


In [71]:
projected_point = adapter.project_point(original_point)
print(projected_point.to_json(indent=2))

{
  "backOffPolicy": 0,
  "OptimisticBackoff___acquireSpinCount": 0.27346229553222656
}


In [72]:
unprojected_point = adapter.unproject_point(projected_point)
print(unprojected_point.to_json(indent=2))

{
  "backOffPolicy": "Optimistic",
  "OptimisticBackoff.acquireSpinCount": 286747
}


In [73]:
# And finally to be sure:
#
assert unprojected_point == original_point